In [1]:
import pandas as pd

In [2]:
metadata = pd.read_csv(r'G:\movies_metadata.csv')
metadata.head()

C:\Users\user\anaconda\envs\py3-TF2.0\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [3]:
c = metadata['vote_average'].mean()

In [4]:
m = metadata['vote_count'].quantile(0.50)
print(m)

10.0


In [5]:
sub_movies = metadata.copy().loc[metadata['vote_average'] >= 6.0]
sub_movies.shape

(24243, 24)

In [6]:
def weighted_rating(x, m=m, c=c):
    v = x['vote_count']
    r = x['vote_average']
    
    return ((v/(v+m) * r) + (m/(m+v) * c))
    

In [7]:
sub_movies['score'] = sub_movies.apply(weighted_rating, axis=1)

sub_movies = sub_movies.sort_values('score', ascending = False)

sub_movies[['title','vote_count','vote_average','score']].head()

,title,vote_count,vote_average,score
10309,Dilwale Dulhania Le Jayenge,661.0,9.1,9.048110
43320,Planet Earth II,50.0,9.5,8.853035
39085,Planet Earth,176.0,8.8,8.628936
314,The Shawshank Redemption,8358.0,8.5,8.496556
834,The Godfather,6024.0,8.5,8.495224


In [8]:
sub_movies['overview'].head()

10309    Raj is a rich, carefree, happy-go-lucky second...
43320    David Attenborough presents a documentary seri...
39085    A documentary miniseries described by its make...
314      Framed in the 1940s for the double murder of h...
834      Spanning the years 1945 to 1955, a chronicle o...
Name: overview, dtype: object

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [10]:
tfvec = TfidfVectorizer(stop_words = 'english')
sub_movies['overview'] = sub_movies['overview'].fillna('')
tf_matrix = tfvec.fit_transform(sub_movies['overview'])
tf_matrix.shape

(24243, 56304)

In [12]:
tfvec.get_feature_names()[4000:4010]

['axis',
 'axl',
 'axmann',
 'axturiax',
 'ay',
 'aya',
 'ayah',
 'ayahuasca',
 'ayaka',
 'ayako']

In [13]:
from sklearn.metrics.pairwise import linear_kernel

In [14]:
cos_sim = linear_kernel(tf_matrix, tf_matrix)

In [15]:
cos_sim.shape

(24243, 24243)

In [16]:
cos_sim[1]

array([0.      , 1.      , 0.080093, ..., 0.      , 0.      , 0.      ])

In [17]:
sub_movies.reset_index(inplace=True, drop=True)
indices = pd.Series(sub_movies.index , index = sub_movies['title']).drop_duplicates()

indices[:10]

title
Dilwale Dulhania Le Jayenge    0
Planet Earth II                1
Planet Earth                   2
The Shawshank Redemption       3
The Godfather                  4
Your Name.                     5
Cosmos                         6
Human                          7
Sansho the Bailiff             8
The Dark Knight                9
dtype: int64

In [18]:
def get_recommendations(title , cos_sim = cos_sim):
    
    index = indices[title]
    
    sim_scores = list(enumerate(cos_sim[index]))
    
    sim_scores = sorted(sim_scores , key = lambda x: x[1], reverse = True)
    
    sim_scores = sim_scores[1:11]
    
    movie_indices = [i[0] for i in sim_scores]
    
    return sub_movies['title'].iloc[movie_indices]

In [22]:
get_recommendations('The Godfather')

16                 The Godfather: Part II
870      The Godfather Trilogy: 1972-1990
2097              The Godfather: Part III
16836                          Blood Ties
6588                             Election
19839                   Short Sharp Shock
17687                  Beck 28 - Familjen
12713                        Violent City
11819                       Live by Night
19742                   The Sign of Venus
Name: title, dtype: object